In [1]:
struct Delivery
    x::Float64
    y::Float64
end;
function random_delivery()
    x = rand(0.0:0.1:100.0)
    y = rand(0.0:0.1:100.0)
    return Delivery(x, y)
end;

struct TSProblem
    deliveries::Array{Delivery}
end;

function random_instance(n_deliveries)
    deliveries = [random_delivery() for _=1:n_deliveries]
    problem = TSProblem(deliveries)
    return problem
end;

function dist(del1::Delivery, del2::Delivery)
    return sqrt((del1.x - del2.x)^2 + (del1.y - del2.y)^2)
end
function calc_travelmatrix(deliveries::Array{Delivery})
    tm = zeros(Float64, length(deliveries), length(deliveries))
    for i = 1:length(deliveries)
        for j = 1:length(deliveries)
            tm[i, j] = dist(deliveries[i], deliveries[j])
        end
    end
    return tm
end


calc_travelmatrix (generic function with 1 method)

In [19]:
using JuMP

import GLPK
import EAGO
import HiGHS

using BenchmarkTools


function basic_mtz(problem, solver, supress_output=false)
    travelmatrix = calc_travelmatrix(problem.deliveries)
    model = Model(solver)
    if supress_output
        set_optimizer_attribute(model, "log_to_console", false)
    end
    # route is an adjence matrix representing a route traveled
    route=@variable(model, route[1:length(problem.deliveries), 1:length(problem.deliveries)], Bin)
    # mtzu is a helper variable to ensure no subtours are allowed (only one continous tour)
    # see MTZ constraint
    mtzu = @variable(model, mtzu[1:length(problem.deliveries)], Int)

    # ensure all events are planned
    @constraint(model, [i = 1:length(problem.deliveries)], sum(route[i, :]) == 1.0)
    # ensure there is just one route
    @constraint(model, [c = 1:length(problem.deliveries)], sum(route[:, c]) == 1.0)
    # disallow traveling to itself
    @constraint(model, [j = 1:length(problem.deliveries)], route[j, j] == 0)

    # MTZ constraints for removing subtours
    n = length(problem.deliveries)
    @constraint(model, [ui = 1:n, uj = 2:n], mtzu[ui] + route[ui, uj] <= mtzu[uj]+ (n - 1) * (1 - route[ui, uj]) )

    traveltime = travelmatrix.* route 
    @objective(model, Min, sum(traveltime))
    optimize!(model)
end

problem=random_instance(5)
basic_mtz(problem, EAGO.Optimizer)
basic_mtz(problem, GLPK.Optimizer)

# nl_solver = optimizer_with_attributes(Ipopt.Optimizer)
# juniper_minlp_solver = optimizer_with_attributes(Juniper.Optimizer, "nl_solver"=>nl_solver)

basic_mtz(problem, HiGHS.Optimizer, true)

In [3]:
problem=random_instance(5)
@time  basic_mtz(problem, GLPK.Optimizer)

  0.000750 seconds (3.33 k allocations: 186.484 KiB)


In [4]:
problem=random_instance(10)
@time  basic_mtz(problem, GLPK.Optimizer)

  0.005408 seconds (8.74 k allocations: 598.047 KiB)


In [5]:
# problem=random_instance(15)
# @time  basic_mtz(problem, GLPK.Optimizer)

In [6]:
# problem=random_instance(20)
# @time  basic_mtz(problem, GLPK.Optimizer)

In [7]:
@benchmark basic_mtz(p, GLPK.Optimizer) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):   2.684 ms … 64.186 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.479 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   22.201 ms ± 18.491 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▃      █     ▃                                              
  ██▁▁▁▁▇▇█▁▁▁▇▁█▁▇▁▇▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▇▁▇▁▁▁▁▁▁▁▁▇▁▇▁▁▁▁▁▁▁▁▁▇ ▁
  2.68 ms         Histogram: frequency by time        64.2 ms <

 Memory estimate: 598.05 KiB, allocs estimate: 8744.

In [8]:
@benchmark basic_mtz(p, EAGO.Optimizer) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):   84.888 ms … 708.156 ms  ┊ GC (min … max): 0.00% … 0.25%
 Time  (median):     221.245 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   271.278 ms ± 186.007 ms  ┊ GC (mean ± σ):  0.03% ± 0.06%

   ▃ █             ▃                  ▃                          
  ▇█▇█▁▁▇▁▁▇▁▇▁▁▁▇▁█▁▁▇▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁█▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▇ ▁
  84.9 ms          Histogram: frequency by time          708 ms <

 Memory estimate: 1.29 MiB, allocs estimate: 23128.

In [16]:
@benchmark basic_mtz(p, HiGHS.Optimizer, true) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):    3.399 ms …    1.037 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      18.381 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   274.959 ms ± 389.529 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▃▁▁▁▃▁▁▁▃▁▁▃ ▁
  3.4 ms           Histogram: frequency by time          1.04 s <

 Memory estimate: 962.17 KiB, allocs estimate: 8692.

In [17]:
using Pkg; Pkg.add("Pavito")

   Resolving package versions...
   Installed Pavito ─ v0.3.5
    Updating `~/.julia/environments/v1.7/Project.toml`
  [cd433a01] + Pavito v0.3.5
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [cd433a01] + Pavito v0.3.5
Precompiling project...
  ✓ Pavito
  1 dependency successfully precompiled in 3 seconds (215 already precompiled)


In [20]:
ENV["CPLEX_STUDIO_BINARIES"] = "/Applications/CPLEX_Studio1210/cplex/bin/x86-64_osx/"

"/Applications/CPLEX_Studio1210/cplex/bin/x86-64_osx/"

In [23]:
import Pkg
Pkg.precompile("CPLEX")
# Pkg.build("CPLEX")

LoadError: MethodError: no method matching precompile(::String)
[0mClosest candidates are:
[0m  precompile(; kwargs...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Pkg/src/API.jl:1011
[0m  precompile([91m::Pkg.Types.Context[39m; internal_call, strict, warn_loaded, already_instantiated, kwargs...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Pkg/src/API.jl:1012